In [5]:
from sklearn.neighbors import KNeighborsClassifier
import os
os.chdir('/Users/ritwikdeshpande/DLH/Project/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
import csv
import numpy as np
import pandas as pd

In [2]:
class KNN:
    def __init__(self, x_train, y_train, x_test, y_test, n):
        self.knn = KNeighborsClassifier(n_neighbors=n)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test

    def train(self):
        self.knn.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.knn.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        return f1_macro, f1_micro

In [3]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']
column_headings = ["Morbidity Class", "KNN1_Macro F1", "KNN1_Micro F1", "KNN5_Macro F1", "KNN5_Micro F1"]

with open("./results/word-embeddings-features/performance_KNN.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2], column_headings[3], column_headings[4]])

In [6]:
all_f1_macro1_scores = []
all_f1_micro1_scores = []

all_f1_macro5_scores = []
all_f1_micro5_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.average(X, axis=1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list1 = []
    f1_micro_list1 = []
    f1_macro_list5 = []
    f1_micro_list5 = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training KNN using TF-IDF Representation
        knn1_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 1)
        knn1_obj.train()

        f1_macro1, f1_micro1 = knn1_obj.test_and_evaluate()

        f1_macro_list1.append(f1_macro1)
        f1_micro_list1.append(f1_micro1)

        knn5_obj = KNN(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold, 5)
        knn5_obj.train()

        f1_macro5, f1_micro5 = knn5_obj.test_and_evaluate()

        f1_macro_list5.append(f1_macro5)
        f1_micro_list5.append(f1_micro5)

    f1_macro1 = np.mean(f1_macro_list1)
    f1_micro1 = np.mean(f1_micro_list1)
    f1_macro5 = np.mean(f1_macro_list5)
    f1_micro5 = np.mean(f1_micro_list5)

    print(f"For n=1, Macro F1 score: {f1_macro1} and Micro F1 Score {f1_micro1}")
    print(f"For n=5, Macro F1 score: {f1_macro5} and Micro F1 Score {f1_micro5}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro1, f1_micro1, f1_macro5, f1_micro5]
    all_f1_macro1_scores.append(f1_macro1)
    all_f1_micro1_scores.append(f1_micro1)

    all_f1_macro5_scores.append(f1_macro5)
    all_f1_micro5_scores.append(f1_micro5)


    with open("./results/word-embeddings-features/performance_KNN.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)

with open("./results/word-embeddings-features/performance_KNN.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([
        sum(all_f1_macro1_scores)/len(all_f1_macro1_scores),  sum(all_f1_micro1_scores)/len(all_f1_micro1_scores),
        sum(all_f1_macro5_scores)/len(all_f1_macro5_scores),  sum(all_f1_micro5_scores)/len(all_f1_micro5_scores) 
                ])
    writer.writerow(row)

Asthma
(572, 3570, 100) (572,) Counter({0.0: 502, 1.0: 70})
For n=1, Macro F1 score: 0.49686271127129 and Micro F1 Score 0.7639745916515426
For n=5, Macro F1 score: 0.47981490964071344 and Micro F1 Score 0.8636418632788869
CAD
(548, 3570, 100) (548,) Counter({1.0: 325, 0.0: 223})
For n=1, Macro F1 score: 0.6361505515229791 and Micro F1 Score 0.654983164983165
For n=5, Macro F1 score: 0.6420510697121621 and Micro F1 Score 0.677003367003367
CHF
(243, 3122, 100) (243,) Counter({1.0: 243})
For n=1, Macro F1 score: 1.0 and Micro F1 Score 1.0
For n=5, Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 3570, 100) (582,) Counter({0.0: 460, 1.0: 122})
For n=1, Macro F1 score: 0.538116567592915 and Micro F1 Score 0.6889246054938634
For n=5, Macro F1 score: 0.5463416086771601 and Micro F1 Score 0.7765926358854471
Diabetes
(567, 3570, 100) (567,) Counter({1.0: 396, 0.0: 171})
For n=1, Macro F1 score: 0.5395344508129 and Micro F1 Score 0.6225877192982455
For n=5, Macro F1 score: 0.57383663